In [ ]:
# HQ simulation for PCDD/Fs and dl-PCBs
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm, gamma, weibull_min, pareto

# Set global font size and resolution
plt.rcParams.update({
    "font.size": 16,           # Set global font size
    "axes.titlesize": 14,      # Title font size
    "axes.labelsize": 16,      # Axis label font size
    "xtick.labelsize": 12,     # X-axis tick font size
    "ytick.labelsize": 12,     # Y-axis tick font size
    "legend.fontsize": 12,     # Legend font size
})

# Parameters
sample_size = 1000  # Number of Monte Carlo simulations
TDI = 0.3  # Tolerable Daily Intake (pg/kg body weight/day)

# Distribution parameters for male and female body weights
body_weight_params = {
    "male": {"mean": 69.6, "sd": 3.4},
    "female": {"mean": 59, "sd": 1.7},
}

# Intake data (g/day)
intake_data = {
    "Egg consumption": {"mean": 38.4, "sd": 38.1},
    "Fish consumption": {"mean": 20.5, "sd": 38.4},
    "Meat consumption": {"mean": 83.9, "sd": 81.1},
}

# Read TEQ concentration data from CSV file
teq_file = "risk_data.csv"
teq_data = pd.read_csv(teq_file)

# Ensure the output folder exists
output_folder = "Risk assessment"
os.makedirs(output_folder, exist_ok=True)

# Print a preview of the loaded data
print("TEQ Data Preview:")
print(teq_data.head())

# Fit a probability distribution to TEQ data
def fit_distribution(data):
    distributions = {
        "normal": norm,
        "lognormal": lognorm,
        "gamma": gamma,
        "weibull": weibull_min,
        "pareto": pareto,
    }

    best_fit = None
    best_aic = float("inf")
    best_params = None

    for name, dist in distributions.items():
        try:
            params = dist.fit(data)
            log_likelihood = np.sum(np.log(np.clip(dist.pdf(data, *params), 1e-10, None)))
            aic = 2 * len(params) - 2 * log_likelihood
            if aic < best_aic:
                best_aic = aic
                best_fit = name
                best_params = params
        except Exception as e:
            continue

    return best_fit, best_params

# Perform Monte Carlo simulation
results = []
all_simulations = {}
combined_hq_simulations = {"male": [], "female": []}

for sample_type in teq_data["Sample Type"].unique():
    hq_simulations = {"male": [], "female": []}

    # Get TEQ concentration for the corresponding sample type
    teq_values = teq_data[teq_data["Sample Type"] == sample_type]["TEQ"]

    # Print sample type and data summary
    print(f"\nProcessing {sample_type}...")
    print(f"TEQ Values for {sample_type}:")
    print(teq_values.describe())

    # Fit the TEQ data to a distribution
    best_fit, best_params = fit_distribution(teq_values)

    # Print the best fit distribution information
    print(f"Best Fit for {sample_type}: {best_fit}, Parameters: {best_params}")

    # Record the best fit model and parameters
    results.append({
        "Sample Type": sample_type,
        "Best Fit": best_fit,
        "Parameters": best_params,
    })

    mapped_type = {
        "egg": "Egg consumption",
        "fish": "Fish consumption",
        "meat": "Meat consumption",
    }[sample_type]

    mean = intake_data[mapped_type]["mean"]
    sd = intake_data[mapped_type]["sd"]

    # Simulate daily intake (g/day) based on normal distribution
    intake_simulated = np.random.normal(mean, sd, sample_size)

    # Simulate TEQ concentration (pg/g) based on the best fit model
    if best_fit == "normal":
        teq_simulated = np.random.normal(*best_params, sample_size)
    elif best_fit == "lognormal":
        teq_simulated = np.random.lognormal(*best_params, sample_size)
    elif best_fit == "gamma":
        shape, loc, scale = best_params
        loc = max(loc, 0)  # Ensure loc is non-negative
        teq_simulated = np.random.gamma(shape, scale, sample_size)
    elif best_fit == "weibull":
        teq_simulated = weibull_min.rvs(*best_params, size=sample_size)
    elif best_fit == "pareto":
        teq_simulated = pareto.rvs(*best_params, size=sample_size)
    else:
        continue

    # Calculate daily intake HQ (HQ = EDI / TDI) for males and females
    for gender, weight_params in body_weight_params.items():
        body_weight_simulated = np.random.normal(weight_params["mean"], weight_params["sd"], sample_size)
        edi = (intake_simulated * teq_simulated) / body_weight_simulated
        hq = edi / TDI
        hq_simulations[gender].append(hq)
        combined_hq_simulations[gender].append(hq)

    # Plot HQ distribution for each food type
    plt.figure(figsize=(10, 6))
    x_values = np.linspace(0, 1.2, 500)  # X-axis range for fitting curves
    for gender, color in zip(["male", "female"], ["#1f77b4", "#ff7f0e"]):  # Optimized colors
        total_hq = np.sum(hq_simulations[gender], axis=0)
        total_hq_positive = total_hq[total_hq >= 0]
        plt.hist(
            total_hq_positive,
            bins=200 if sample_type == "egg" else 200,  # Use smaller bins for eggs
            alpha=0.6,
            density=True,
            label=f"{sample_type.capitalize()} - {gender.capitalize()}",
            color=color
        )
        # Fit curves and mark percentiles
        if best_fit == "normal":
            fitted_curve = norm.pdf(x_values, *norm.fit(total_hq_positive))
        elif best_fit == "lognormal":
            fitted_curve = lognorm.pdf(x_values, *lognorm.fit(total_hq_positive))
        elif best_fit == "gamma":
            fitted_curve = gamma.pdf(x_values, *gamma.fit(total_hq_positive))
        elif best_fit == "weibull":
            fitted_curve = weibull_min.pdf(x_values, *weibull_min.fit(total_hq_positive))
        elif best_fit == "pareto":
            fitted_curve = pareto.pdf(x_values, *pareto.fit(total_hq_positive))
        else:
            fitted_curve = None

        if fitted_curve is not None:
            plt.plot(x_values, fitted_curve, color=color, linestyle='--', linewidth=2, label=f"{gender.capitalize()} Fit Curve")

        p5 = np.percentile(total_hq_positive, 5)
        p50 = np.percentile(total_hq_positive, 50)
        p95 = np.percentile(total_hq_positive, 95)
        mean_val = np.mean(total_hq_positive)

        plt.axvline(p5, color=color, linestyle='dashed', linewidth=1, label=f"{gender.capitalize()} 5%")
        plt.text(p5, plt.ylim()[1] * 0.8, f"{p5:.3f}", color="black", rotation=90)

        plt.axvline(p50, color=color, linestyle='solid', linewidth=1, label=f"{gender.capitalize()} Median")
        plt.text(p50, plt.ylim()[1] * 0.6, f"{p50:.3f}", color="black", rotation=90)

        plt.axvline(p95, color=color, linestyle='dotted', linewidth=1, label=f"{gender.capitalize()} 95%")
        plt.text(p95, plt.ylim()[1] * 0.4, f"{p95:.3f}", color="black", rotation=90)

        plt.axvline(mean_val, color=color, linestyle='dashdot', linewidth=1, label=f"{gender.capitalize()} Mean")
        plt.text(mean_val, plt.ylim()[1] * 0.2, f"{mean_val:.3f}", color="black", rotation=90)

    plt.title(f"HQ Distribution for {sample_type.capitalize()} (Male vs Female)")
    plt.xlabel("HQ (EDI / TDI)")
    plt.ylabel("Probability Density")
    plt.xlim(0, 0.5 if sample_type == "egg" else 1.2)  # Adjust X-axis range for eggs
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"{sample_type}_hq_distribution_gender.png"), dpi=300)
    plt.close()

# Calculate combined HQ for all food types
combined_hq = {gender: np.sum(combined_hq_simulations[gender], axis=0) for gender in ["male", "female"]}

# Plot combined HQ distribution for males and females
plt.figure(figsize=(10, 6))
x_values = np.linspace(0, 1.8, 500)  # X-axis range for fitting curves
for gender, color in zip(["male", "female"], ["#1f77b4", "#ff7f0e"]):  # Optimized colors
    combined_hq_positive = combined_hq[gender][combined_hq[gender] >= 0]
    plt.hist(combined_hq_positive, bins=300, alpha=0.7, density=True, label=f"{gender.capitalize()}", color=color)

    p5 = np.percentile(combined_hq_positive, 5)
    p50 = np.percentile(combined_hq_positive, 50)
    p95 = np.percentile(combined_hq_positive, 95)
    mean_val = np.mean(combined_hq_positive)

    plt.axvline(p5, color=color, linestyle='dashed', linewidth=1, label=f"{gender.capitalize()} 5%")
    plt.text(p5, plt.ylim()[1] * 0.8, f"{p5:.3f}", color="black", rotation=90)

    plt.axvline(p50, color=color, linestyle='solid', linewidth=1, label=f"{gender.capitalize()} Median")
    plt.text(p50, plt.ylim()[1] * 0.6, f"{p50:.3f}", color="black", rotation=90)

    plt.axvline(p95, color=color, linestyle='dotted', linewidth=1, label=f"{gender.capitalize()} 95%")
    plt.text(p95, plt.ylim()[1] * 0.4, f"{p95:.3f}", color="black", rotation=90)

    plt.axvline(mean_val, color=color, linestyle='dashdot', linewidth=1, label=f"{gender.capitalize()} Mean")
    plt.text(mean_val, plt.ylim()[1] * 0.2, f"{mean_val:.3f}", color="black", rotation=90)

plt.title("HQ Distribution for Combined (Male vs Female)")
plt.xlabel("HQ (EDI / TDI)")
plt.ylabel("Probability Density")
plt.xlim(0, 1.8)  # Adjust X-axis max to 1.8
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "combined_hq_distribution_gender.png"), dpi=300)
plt.close()


In [ ]:
# 1/MOE simulation for PBDEs
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm, gamma, weibull_min, pareto

# Set global font size and resolution
plt.rcParams.update({
    "font.size": 12,
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
})

# Parameters
sample_size = 1000  # Number of Monte Carlo simulations
#TDI = 0.3  # Tolerable Daily Intake (pg/kg body weight/day)

# Chronic dietary exposure values provided by EFSA, converted to pg/kg b.w.
drh_values = {
    "BDE-47": 172 * 1000,
    "BDE-99": 4.2 * 1000,
    "BDE-153": 9.6 * 1000,
    "BDE-209": 1700 * 1000
}

# Male and female body weight distribution parameters
body_weight_params = {
    "male": {"mean": 69.6, "sd": 3.4},
    "female": {"mean": 59, "sd": 1.7},
}

# Food intake data (g/day)
intake_data = {
    "Egg consumption": {"mean": 38.4, "sd": 38.1},
    "Fish consumption": {"mean": 20.5, "sd": 38.4},
    "Meat consumption": {"mean": 83.9, "sd": 81.1},
}

# Read concentration data from a CSV file
input_file = "risk_data.csv"
data = pd.read_csv(input_file)

# Ensure the output folder exists
output_folder = "Risk assessment"
os.makedirs(output_folder, exist_ok=True)

# Distribution mapping
distribution_mapping = {
    "normal": norm,
    "lognormal": lognorm,
    "gamma": gamma,
    "weibull": weibull_min,
    "pareto": pareto,
}

# Fit a probability distribution to concentration data
def fit_distribution(data):
    best_fit = None
    best_statistic = float("inf")
    best_params = None

    for name, dist in distribution_mapping.items():
        try:
            params = dist.fit(data)
            statistic = np.sum(np.abs(data - dist.mean(*params)))
            if statistic < best_statistic:
                best_statistic = statistic
                best_fit = name
                best_params = params
        except Exception as e:
            continue

    if best_fit is None:
        return None, None

    return best_fit, best_params

# Process each BDE
for bde in ["BDE-47", "BDE-99", "BDE-153", "BDE-209"]:
    moe_simulations = {"male": [], "female": []}

    for sample_type, intake_info in intake_data.items():
        bde_values = data[(data["Sample Type"] == sample_type.split()[0].lower()) & (data[bde] > 0)][bde]

        if bde_values.empty:
            print(f"No valid data for {bde} in {sample_type}. Skipping.")
            continue

        # Fit concentration data to a distribution
        best_fit, best_params = fit_distribution(bde_values)
        if best_fit is None:
            print(f"Warning: No suitable distribution found for {bde} in {sample_type}.")
            continue

        # Simulate EDI distribution
        intake_simulated = np.random.normal(loc=intake_info["mean"], scale=intake_info["sd"], size=sample_size)
        concentration_simulated = distribution_mapping[best_fit].rvs(*best_params, size=sample_size)

        for gender in ["male", "female"]:
            body_weight_mean = body_weight_params[gender]["mean"]
            edi = concentration_simulated * intake_simulated / body_weight_mean
            inverse_moe = edi / drh_values[bde]
            moe_simulations[gender].append(inverse_moe)

    # Combine all simulation results and save
    plt.figure(figsize=(10, 6))
    colors = {"male": "blue", "female": "red"}

    for gender in ["male", "female"]:
        total_moe = np.concatenate(moe_simulations[gender])
        total_moe = total_moe[total_moe >= 0]  # Keep only non-negative values
        
        # Plot histogram
        plt.hist(
            total_moe, bins=300, density=True, alpha=0.5, 
            color=colors[gender], label=f"{gender.capitalize()} Histogram"
        )
        
        # Fit kernel density distribution curve
        best_fit, best_params = fit_distribution(total_moe)
        if best_fit:
            x = np.linspace(0, 0.001, 500)
            pdf = distribution_mapping[best_fit].pdf(x, *best_params)
            plt.plot(x, pdf, color=colors[gender], label=f"{gender.capitalize()} Fit ({best_fit})")
        
        # Calculate statistics and draw vertical lines
        mean_val = np.mean(total_moe)
        p5, p50, p95 = np.percentile(total_moe, [5, 50, 95])
        
        plt.axvline(p5, color=colors[gender], linestyle='dashed', linewidth=1, label=f"{gender.capitalize()} 5%")
        plt.text(p5, plt.ylim()[1] * 0.8, f"{p5:.3f}", color="black", rotation=90)
        
        plt.axvline(p50, color=colors[gender], linestyle='solid', linewidth=1, label=f"{gender.capitalize()} Median")
        plt.text(p50, plt.ylim()[1] * 0.6, f"{p50:.3f}", color="black", rotation=90)
        
        plt.axvline(p95, color=colors[gender], linestyle='dotted', linewidth=1, label=f"{gender.capitalize()} 95%")
        plt.text(p95, plt.ylim()[1] * 0.4, f"{p95:.3f}", color="black", rotation=90)
        
        plt.axvline(mean_val, color=colors[gender], linestyle='dashdot', linewidth=1, label=f"{gender.capitalize()} Mean")
        plt.text(mean_val, plt.ylim()[1] * 0.2, f"{mean_val:.3f}", color="black", rotation=90)
        
        # Save as CSV file
        output_path = os.path.join(output_folder, f"{bde}_{gender}_MOE_Distribution.csv")
        pd.DataFrame(total_moe, columns=["1/MOE"]).to_csv(output_path, index=False)

    # Chart adjustments
    plt.title(f"1/MOE Distribution for {bde} (Male & Female)")
    plt.xlabel("1/MOE")
    plt.ylabel("Density")
    plt.xlim(0, 0.001)  # Limit the maximum X-axis value to 0.001
    plt.legend()
    plt.grid()
    
    # Save image
    plt_path_combined = os.path.join(output_folder, f"{bde}_combined_MOE_Distribution.png")
    plt.savefig(plt_path_combined)
    plt.close()

    print(f"{bde} combined 1/MOE distribution plot saved to {plt_path_combined}")
